In [18]:
import pandas as pd

### Этап 1. Получение данных

In [28]:
df = pd.read_csv('datasets/music_project.csv')

In [29]:
df.head()

,Unnamed: 0,userID,Track,artist,genre,City,time,Day
0,0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [30]:
df.tail()

,Unnamed: 0,userID,Track,artist,genre,City,time,Day
65074,65074,729CBB09,My Name,McLean,rnb,Moscow,13:32:28,Wednesday
65075,65075,D08D4A55,Maybe One Day (feat. Black Spade),Blu & Exile,hip,Saint-Petersburg,10:00:00,Monday
65076,65076,C5E3A0D5,Jalopiina,NaN,industrial,Moscow,20:09:26,Friday
65077,65077,321D0506,Freight Train,Chas McDevitt,rock,Moscow,21:43:59,Friday
65078,65078,3A64EF84,Tell Me Sweet Little Lies,Monica Lopez,country,Moscow,21:59:46,Friday


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 8 columns):
Unnamed: 0    65079 non-null int64
  userID      65079 non-null object
Track         63848 non-null object
artist        57876 non-null object
genre         63881 non-null object
  City        65079 non-null object
time          65079 non-null object
Day           65079 non-null object
dtypes: int64(1), object(7)
memory usage: 4.0+ MB


**Полученная информация**: 
В срезе 8 столбцов тип данных int64 (ненужный столбец) и object и 65079 строк

In [32]:
# Удаляем столбец или так del df['column_name'] или
df.drop('Unnamed: 0', axis=1, inplace=True)

In [34]:
df.sample(1)

,userID,Track,artist,genre,City,time,Day
27704,E9B4A038,Boş Bardak,Fettah Can,world,Saint-Petersburg,09:54:28,Wednesday


Информация в столбцах

* userID — идентификатор пользователя;
* Track — название трека;  
* artist — имя исполнителя;
* genre — название жанра;
* City — город, в котором происходило прослушивание;
* time — время, в которое пользователь слушал трек;
* Day — день недели.

Проблема: Количество значений в столбцах разное (пропущены значения) и пробелы в названиях столбцов. Для проверки рабочих гипотез особенно ценны столбцы time, day и City. Данные из столбца *ganre* позволят узнать самые популярные жанры.

### Этап 2. Предобработка данных
- убрать пропуски в названиях столбцов
- переименовать столбцы
- проверка на пропуски значений
- проверка на дубликаты

In [36]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [37]:
df.set_axis(['user_id', 'track_name', 'artist_name', 
             'genre_name', 'city', 'time', 'weekday'], axis='columns', inplace=True)

In [38]:
df.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

**Проверка на дубликаты**

In [39]:
df.isnull().sum()

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

In [40]:
df.isna().sum()

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

Пустые значения свидетельствуют, что для некоторых треков доступна не вся информация. Причины могут быть разные: скажем, не назван конкретный исполнитель народной песни. Хуже, если проблемы с записью данных. Каждый отдельный случай необходимо разобрать и выявить причину.

Заменяем пропущенные значения в столбцах с названием трека и исполнителя на строку 'unknown'. После этой операции нужно убедиться, что таблица больше не содержит пропусков.

In [42]:
df['track_name'] = df['track_name'].fillna('unknown')

In [44]:
df['artist_name'] = df['artist_name'].fillna('unknown')

In [45]:
df.isnull().sum()

user_id           0
track_name        0
artist_name       0
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

Удаляем в столбце с жанрами пустые значения; убеждаемся, что их больше не осталось.

In [47]:
df.dropna(subset = ['genre_name'], inplace=True)

In [48]:
df.isnull().sum()

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64

In [50]:
df.duplicated().sum()

3755

In [52]:
df = df.drop_duplicates().reset_index(drop=True)

Дубликаты могли появиться вследствие сбоя в записи данных. Стоит обратить внимание и разобраться с причинами

In [53]:
df.duplicated().sum()

0

Сохраняем список уникальных значений столбца с жанрами в переменной genres_list.

Объявим функцию find_genre() для поиска неявных дубликатов в столбце с жанрами. Например, когда название одного и того же жанра написано разными словами.

In [59]:
genres_list = df['genre_name'].unique()

In [60]:
genres_list

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'alternative', 'children', 'rnb', 'hip', 'jazz',
       'postrock', 'latin', 'classical', 'metal', 'reggae', 'tatar',
       'blues', 'instrumental', 'rusrock', 'dnb', 'türk', 'post',
       'country', 'psychedelic', 'conjazz', 'indie', 'posthardcore',
       'local', 'avantgarde', 'punk', 'videogame', 'techno', 'house',
       'christmas', 'melodic', 'caucasian', 'reggaeton', 'soundtrack',
       'singer', 'ska', 'shanson', 'ambient', 'film', 'western', 'rap',
       'beats', "hard'n'heavy", 'progmetal', 'minimal', 'contemporary',
       'new', 'soul', 'holiday', 'german', 'tropical', 'fairytail',
       'spiritual', 'urban', 'gospel', 'nujazz', 'folkmetal', 'trance',
       'miscellaneous', 'anime', 'hardcore', 'progressive', 'chanson',
       'numetal', 'vocal', 'estrada', 'russian', 'classicmetal',
       'dubstep', 'club', 'deep', 'southern', 'black', 'folkrock',
       'fitness', 'french', 'd

In [62]:
def find_genre(genre_name):
    counter = 0
    for i in range(len(genres_list)):
        if genres_list[i] == genre_name:
            counter += 1
    return counter

Правильное название — *hiphop*. Поищем другие варианты:

* hip
* hop
* hip-hop

In [63]:
find_genre('hip')

1

In [64]:
find_genre('hop')

0

In [65]:
find_genre('hip-hop')

0

Объявим функцию *find_hip_hop()*, которая заменяет неправильное название этого жанра в столбце *'genre_name'* на *'hiphop'* и проверяет успешность выполнения замены.

Так исправляем все варианты написания, которые выявила проверка.

In [69]:
def find_hip_hop(df, wrong):
    correct = 'hiphop'
    df['genre_name'] = df['genre_name'].replace(wrong, correct)
    wrong_count = df[df['genre_name'] == wrong]['genre_name'].count()
    return wrong_count

In [70]:
find_hip_hop(df, 'hip')

0

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60126 entries, 0 to 60125
Data columns (total 7 columns):
user_id        60126 non-null object
track_name     60126 non-null object
artist_name    60126 non-null object
genre_name     60126 non-null object
city           60126 non-null object
time           60126 non-null object
weekday        60126 non-null object
dtypes: object(7)
memory usage: 3.2+ MB


**Вывод:**
На этапе предобработки в данных обнаружились не только пропуски и проблемы с названиями столбцов, но и всяческие виды дубликатов. Их удаление позволит провести анализ точнее. Поскольку сведения о жанрах важно сохранить для анализа, не просто удаляем все пропущенные значения, но заполним пропущенные имена исполнителей и названия треков. Имена столбцов теперь корректны и удобны для дальнейшей работы.

### 3. Анализ данных
- прослушивание по дням недели в разных городах
- жанры по дням недели в определенное время

Была выдвинута гипотеза, что в Москве и Санкт-Петербурге пользователи слушают музыку по-разному. Проверяем это предположение по данным о трёх днях недели — ```понедельнике, среде и пятнице.```

Для каждого города устанавливаем количество прослушанных  в эти дни композиций с известным жанром, и сравниваем результаты.

Группируем данные по городу и вызовом метода count() подсчитываем композиции, для которых известен жанр.

In [74]:
df.sample()

,user_id,track_name,artist_name,genre_name,city,time,weekday
11263,B0FB732C,Tell Me,Take/Five,electronic,Saint-Petersburg,14:51:30,Monday


In [73]:
df.groupby('city')['genre_name'].count()

city
Moscow              41892
Saint-Petersburg    18234
Name: genre_name, dtype: int64

В Москве прослушиваний больше, чем в Питере, но это не значит, что Москва более активна. У Яндекс.Музыки в целом больше пользователей в Москве, поэтому величины сопоставимы.

Сгруппируем данные по дню недели

In [76]:
df.groupby('weekday')['genre_name'].count()

weekday
Friday       21482
Monday       20866
Wednesday    17778
Name: genre_name, dtype: int64

Понедельник и пятница — время для музыки; по средам пользователи немного больше вовлечены в работу.

Создаём функцию number_tracks(), которая принимает как параметры таблицу, день недели и название города, а возвращает количество прослушанных композиций, для которых известен жанр. Проверяем количество прослушанных композиций для каждого города и понедельника, затем среды и пятницы.

In [85]:
def number_tracks(df, day, city):
    track_list = df[(df['weekday'] == day) & (df['city'] == city)]
    track_list_count = track_list['genre_name'].count()
    return track_list_count

In [86]:
# <список композиций для Москвы в понедельник>
number_tracks(df, 'Monday', 'Moscow')

15347

In [87]:
number_tracks(df, 'Wednesday', 'Moscow')

10865

In [88]:
number_tracks(df, 'Friday', 'Moscow')

15680

In [89]:
number_tracks(df, 'Monday', 'Saint-Petersburg')

5802

In [90]:
number_tracks(df, 'Wednesday', 'Saint-Petersburg')

6913

In [91]:
number_tracks(df, 'Friday', 'Saint-Petersburg')

5802

In [96]:
data = [['Moscow', 15347, 10865, 15680], ['Saint-Petersburg', 5802, 6913, 5802]]
columns = ['city', 'Monday', 'Wednesday', 'Friday']
table = pd.DataFrame(data=data, columns=columns)
table

,city,Monday,Wednesday,Friday
0,Moscow,15347,10865,15680
1,Saint-Petersburg,5802,6913,5802


,city,Monday,Wednesday,Friday
0,Moscow,15347,10865,15680
1,Saint-Petersburg,5802,6913,5802


**Вывод**: результаты показывают, что относительно среды музыку в Петербурге и Москве слушают «зеркально»: в Москве пики приходятся на понедельник и пятницу, а в среду время прослушивания снижается. Тогда как в Санкт-Петербурге среда — день самого большого интереса к музыке, а в понедельник и пятницу он меньше, причём почти одинаково меньше.

### Утро понедельника и вечер пятницы — разная музыка или одна и та же?

Ищем ответ на вопрос, какие жанры преобладают в разных городах в понедельник утром и в пятницу вечером. Есть предположение, что в понедельник утром пользователи слушают больше бодрящей музыки (например, жанра поп), а вечером пятницы — больше танцевальных (например, электронику).

Получим таблицы данных по Москве **moscow_general** и по Санкт-Петербургу **spb_general.**

In [101]:
moscow_general = df[df['city'] == 'Moscow']
moscow_general.head(1)

,user_id,track_name,artist_name,genre_name,city,time,weekday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday


In [102]:
spb_general = df[df['city'] == 'Saint-Petersburg']
spb_general.tail(1)

,user_id,track_name,artist_name,genre_name,city,time,weekday
60122,D08D4A55,Maybe One Day (feat. Black Spade),Blu & Exile,hiphop,Saint-Petersburg,10:00:00,Monday


Создаём функцию **genre_weekday()**, которая возвращает список жанров по запрошенному дню недели и времени суток с такого-то часа по такой-то.

In [122]:
def genre_weekday(df, day, time1, time2):
    genre_list = df[(df['weekday'] == day) & (df['time'] >= time1) & (df['time'] <= time2)]
    genre_list_sorted = genre_list.sort_values(by = 'genre_name', ascending = False).head(10)
    return genre_list_sorted

In [123]:
genre_weekday(moscow_general, 'Monday', '07:00:00', '10:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
27442,D7FB50DA,Drumming Circle,Professor Trance,worldbeat,Moscow,09:30:47,Monday
28305,1F8D79D4,Tavadjanutyun (Armenian),Robert Chilingirian,world,Moscow,08:22:00,Monday
14673,76E2BBA6,Sarut Femeie Mana Ta,Ian Raiburg,world,Moscow,08:01:55,Monday
13781,755B065B,Bilan,Espoir 2000,world,Moscow,08:24:21,Monday
37761,FCC1AE1A,Довбуш-Казак,Ukrainska Selska Orchestra,world,Moscow,09:26:17,Monday
37684,9BBA5C98,Como Te Sentes Tu ?,Ary,world,Moscow,08:14:23,Monday
37675,755B065B,Bilan,Koffi Olomide,world,Moscow,08:24:11,Monday
37617,2D0E6AA,The Song of the Butterfly,Estas Tonne Istvan Sky Kek Eg Pablo Arellano &...,world,Moscow,09:45:12,Monday
32699,EE1569CA,Dai Din Bull Ca Raj Kapoor,Laura Vass,world,Moscow,09:20:24,Monday
37583,FB8A3EA8,Продажная любовь,Мурат Тхагалегов,world,Moscow,09:36:11,Monday


In [117]:
genre_weekday(spb_general, 'Monday', '07:00:00', '10:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
40877,E748BD8F,Саморазвитие личности,йога,world,Saint-Petersburg,09:42:30,Monday
50732,2F879AEC,Sara,Elji Beatzkilla,world,Saint-Petersburg,09:47:40,Monday
15343,D6A17B0A,Balayo,Samatar & Kasbah Rockers,world,Saint-Petersburg,09:16:09,Monday
30068,692AEC35,Baiana,Emicida,world,Saint-Petersburg,09:24:38,Monday
45091,B5B6FE0A,Sade,Badmos,world,Saint-Petersburg,09:03:38,Monday
42316,6B62E9CE,The Color of The Night,L' Amore Orchestra,world,Saint-Petersburg,08:56:04,Monday
21206,F33FA8C,Pivni Tkach posh edit light,Katya Chilly,world,Saint-Petersburg,08:51:27,Monday
54076,26A54BF,Niz Tone,Toke-Cha,world,Saint-Petersburg,08:42:23,Monday
15975,FAF7BEE6,Smoke gets in your eyes,Jazz Orchestra,world,Saint-Petersburg,09:00:50,Monday
20813,4D2A996C,Closer to Heaven,Mysteria,world,Saint-Petersburg,09:30:23,Monday


In [130]:
genre_weekday(moscow_general, 'Friday', '19:00:00', '22:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
54275,2508CEE3,Humee Hum Brahm Hum,Siri Sadhana Kaur,world,Moscow,21:20:28,Friday
42379,1EAECCC2,Awakening,Magic Carpet,world,Moscow,21:48:26,Friday
42298,54F93A25,Franc Jeu,Medhy Custos,world,Moscow,21:41:11,Friday
32383,334FBD28,Ты ушла,Руслан Исаков,world,Moscow,21:15:28,Friday
1847,B13DB6EB,Chim Chim Cher-ee,Dream House,world,Moscow,20:42:46,Friday
32408,983CB455,Deo's Erotas,Daemonia Nymphe,world,Moscow,20:08:45,Friday
14990,1DCC8101,Pajaros,Leonardo Trincabelli,world,Moscow,21:22:49,Friday
52123,77CBF89C,Oh My Brothers,Castillo Nasheeds,world,Moscow,20:16:41,Friday
42221,3D05400D,Шальная осень,Александр Кальянов,world,Moscow,20:14:19,Friday
32743,F488ADA5,The Mayqueen,Kevin Budd,world,Moscow,21:49:21,Friday


In [131]:
genre_weekday(spb_general, 'Friday', '19:00:00', '22:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
30832,238848EA,The Chicken And The Hawk,DADDY'S REBELS,world,Saint-Petersburg,20:36:58,Friday
53528,2DFAA635,Nani,Heart Tones feat. Tutarchela,world,Saint-Petersburg,20:18:23,Friday
1204,47E5088,Как ты там,Катерина Голицина,world,Saint-Petersburg,21:02:08,Friday
50863,F3FB8127,Rayare Unnai Thedi,Jey Raggaveindra,world,Saint-Petersburg,20:52:01,Friday
2436,3581900D,Azerix,Anna Rf,world,Saint-Petersburg,21:33:21,Friday
51237,7E67C184,Sky Dancer / Kandroma (Tibetan Prayer to Honor...,unknown,world,Saint-Petersburg,20:00:15,Friday
40653,6878926B,Kolomeika,Klezgoyim,world,Saint-Petersburg,21:48:38,Friday
18013,1D507F3F,A Cochabamba Me Voy,Daniele Sepe,world,Saint-Petersburg,21:50:57,Friday
40083,D49F3360,Ganar,The Pelos,world,Saint-Petersburg,21:12:36,Friday
39976,529A338E,"Ramy Sabry ""Ta'ali""",DJ Nabil,world,Saint-Petersburg,21:21:07,Friday


In [ ]:
# второй вариант решения 
def genre_weekday2(df, day, time1, time2):
    genre_list2 = df[(df['weekday'] == day)]  
    genre_list_sorted2 = genre_list2.groupby('genre_name')['genre_name'].count().sort_values(ascending=False).head(10)
    return genre_list_sorted2

In [126]:
# вся выборка по жанрам
genre_weekday2(moscow_general, 'Monday', '07:00:00', '10:00:00')

genre_name
pop            2154
dance          1669
rock           1452
electronic     1432
hiphop          789
classical       558
world           548
alternative     499
ruspop          478
rusrap          440
Name: genre_name, dtype: int64

In [127]:
genre_weekday2(spb_general, 'Monday', '07:00:00', '10:00:00')

genre_name
pop            732
dance          589
rock           577
electronic     523
hiphop         277
alternative    199
classical      187
jazz           174
rusrap         164
world          163
Name: genre_name, dtype: int64

In [129]:
genre_weekday2(moscow_general, 'Friday', '19:00:00', '22:00:00')

genre_name
pop            2166
dance          1602
rock           1497
electronic     1430
hiphop          814
classical       603
ruspop          538
world           537
alternative     524
rusrap          441
Name: genre_name, dtype: int64

In [128]:
genre_weekday2(spb_general, 'Friday', '19:00:00', '22:00:00')

genre_name
pop            748
dance          639
rock           593
electronic     556
hiphop         329
classical      222
alternative    201
rusrap         173
ruspop         168
world          157
Name: genre_name, dtype: int64


Популярные жанры в понедельник утром в Питере и Москве оказались похожи: везде, как и предполагалось, популярен поп. Несмотря на это, концовка топ-10 для двух городов различается: в Питере в топ-10 входит джаз и русский рэп, а в Москве жанр world.

В конце недели ситуация не меняется. Поп-музыка всё так же на первом месте. Опять разница заметна только в концовке топ-10, где в Питере пятничным вечером тоже присутствует жанр world.

**Вывод**: Жанр поп безусловный лидер, а топ-5 в целом не различается в обеих столицах. При этом видно, что концовка списка более «живая»: для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени.

### Москва и Питер — две разные столицы, два разных направления в музыке. Правда?

Гипотеза: Питер богат своей рэп-культурой, поэтому это направление там слушают чаще, а Москва — город контрастов, но основная масса пользователей слушает попсу.

In [146]:
moscow_general.head(1)

,user_id,track_name,artist_name,genre_name,city,time,weekday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday


In [143]:
moscow_general['genre_name'].value_counts().head(10)

pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre_name, dtype: int64

In [144]:
spb_general['genre_name'].value_counts().head(10)

pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre_name, dtype: int64

**Вывод**: В Москве, кроме абсолютно популярного жанра поп, есть направление русской популярной музыки. Значит, что интерес к этому жанру шире. А рэп, вопреки предположению, занимает в обоих городах близкие позиции.

### Этап 4. Результаты исследования

Рабочие гипотезы:

* музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;

* списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;

* население двух городов предпочитает разные музыкальные жанры.

**Общие результаты**

Москва и Петербург сходятся во вкусах: везде преобладает популярная музыка. При этом зависимости предпочтений от дня недели в каждом отдельном городе нет — люди постоянно слушают то, что им нравится. Но между городами в разрезе дней неделей наблюдается зеркальность относительно среды: Москва больше слушает в понедельник и пятницу, а Петербург наоборот - больше в среду, но меньше в понедельник и пятницу.

В результате первая гипотеза подтверждена, вторая гипотеза не подтверждена и третья не подтверждена.